In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score

#read in files and make necessary adjustments
df1 = pd.read_csv('.csv')
df1 = df1.fillna(0)
df1 = df1.drop(df1.index[20:1000])
df1 = df1.drop(df1.columns[[37]], axis=1)
#print(df1)

df2 = pd.read_csv('.csv')
df2 = df2.fillna(0)
df2 = df2.drop(df2.index[20:1000])
#df2 = df2.drop(df2.columns[[38]], axis=1)
#print(df2)


df1.rename(columns=lambda x: x.strip(), inplace=True)
df2.rename(columns=lambda x: x.strip(), inplace=True)


df_compare = df1.compare(df2, align_axis=1, keep_shape=True, keep_equal=True)

# style func to highlight the cells that are different
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('self', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

df_compare2 = df_compare[(df1 != df2).any(1)].style.apply(highlight_diff, axis=None)
writer = pd.ExcelWriter('3_Irene_P12_comparison_jw.xlsx', engine='xlsxwriter')
df_compare2.to_excel(writer, sheet_name='Differences')
writer.close()  
 

# COHEN KAPPA VALUES
arrayColumns = df1.columns
arrayr1all = []
arrayr2all = []

for x in range(3, 37):
    arrayr1all.append(df1[arrayColumns[x]].tolist())
    arrayr2all.append(df2[arrayColumns[x]].tolist())

data = []
arrayColumns = arrayColumns[3:]

for i in range(len(arrayColumns)):
    data.append([arrayColumns[i], cohen_kappa_score(arrayr1all[i], arrayr2all[i])])

summary = pd.DataFrame(data)
summary.to_csv('test_kappa_jw', sep='\t', index=False)


/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_d